# Libs

In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# load data

In [2]:
import datetime
from helpers import *
from validation import *
from implementations import *

In [10]:
data_path = "./data/train.csv"
yb, tx, ids = load_csv_data(data_path, sub_sample=False)  # test label, test feature, test id or idx number

In [11]:
test_data = load_csv_data_general(data_path)

# display

In [5]:
print(tx.shape)
std_data = np.std(tx,0)
mean_data = np.mean(tx,0)
print(std_data.shape, mean_data.shape)
print(std_data,'\n',mean_data)

(250000, 30)
(30,) (30,)
[406.34483401  35.34481492  40.82860887  63.65555431 454.47965615
 657.97098617 453.01897051   0.78290955  22.2734492  115.70588372
   0.84474126   1.19358245 453.59581401  22.41203584   1.21407622
   1.81675941  22.06487828   1.26497962   1.81660763  32.8946274
   1.81221908 126.49925272   0.97742435 532.96172343 489.33730734
 489.33290465 479.87453609 453.38371728 453.3881105   98.01546598] 
 [-4.90230794e+01  4.92398193e+01  8.11819816e+01  5.78959617e+01
 -7.08420675e+02 -6.01237051e+02 -7.09356603e+02  2.37309984e+00
  1.89173324e+01  1.58432217e+02  1.43760943e+00 -1.28304708e-01
 -7.08985189e+02  3.87074191e+01 -1.09730480e-02 -8.17107200e-03
  4.66602072e+01 -1.95074680e-02  4.35429640e-02  4.17172345e+01
 -1.01191920e-02  2.09797178e+02  9.79176000e-01 -3.48329567e+02
 -3.99254314e+02 -3.99259788e+02 -6.92381204e+02 -7.09121609e+02
 -7.09118631e+02  7.30645914e+01]


## preprocess

In [16]:
median_data=np.median(tx,0)

In [7]:
tx_clean = tx
for w1 in range(tx.shape[1]): 
        for w0 in range(tx.shape[0]):
            if tx[w0,w1] == -999:
               tx_clean[w0,w1] = median_data[w1]

np.std(tx_clean,0)            

array([ 53.09625203,  35.34481492,  40.82860887,  63.65555431,
       454.47965615, 657.97098617, 453.01897051,   0.78290955,
        22.2734492 , 115.70588372,   0.84474126,   1.19358245,
       453.59581401,  22.41203584,   1.21407622,   1.81675941,
        22.06487828,   1.26497962,   1.81660763,  32.8946274 ,
         1.81221908, 126.49925272,   0.97742435,  52.09478013,
         1.65823006,   1.73573955, 479.87453609, 453.38371728,
       453.3881105 ,  98.01546598])

In [8]:
tx_clean2 = pre_process_data(tx)
np.std(tx_clean2,0)  

TypeError: pre_process_data() missing 1 required positional argument: 'ignore_cols'

## preprocess normalization

In [9]:
txn = normalize_data(tx_clean2)

NameError: name 'tx_clean2' is not defined

## preprocess jets

In [6]:
x_tr0, x_tr1, x_tr2, x_tr3, x_tr0_wm, x_tr1_wm, x_tr2_wm, x_tr3_wm = pre_process_data_jets(test_data)

print(tx.shape)    
print(x_tr0.shape)
print(x_tr1.shape)
print(x_tr2.shape)
print(x_tr3.shape)
print(x_tr0_wm.shape)
print(x_tr1_wm.shape)
print(x_tr2_wm.shape)
print(x_tr3_wm.shape)

(250000, 30)
(73790, 23)
(69982, 26)
(47427, 32)
(20687, 32)
(26123, 22)
(7562, 25)
(2952, 31)
(1477, 31)


## feature augmentation 

In [20]:
#txn = normalize_data(tx)
#print(txn.shape)

def build_poly_cross_terms(x, degree, ct=False, sqrt=False, square=False):
    n_x = len(x)
    nbr_param = len(x[0])
    #print(nbr_param)
    nbr_ct = 0
    nbr_ct += int(nbr_param * (nbr_param - 1) / 2)
    mat = np.zeros((n_x, (degree + 1) * nbr_param + nbr_ct))
    
    mat[:, :nbr_param] = x
    idx = nbr_param
    for l in range(nbr_ct):
        for m in range(l + 1, nbr_param):
            mat[:, idx] = x[:, l] * x[:, m]
            idx += 1
                
    return mat

x_tr0_augment = build_poly_cross_terms(x_tr0,0,False,False,False)
x_tr1_augment = build_poly_cross_terms(x_tr1,0,False,False,False)
x_tr2_augment = build_poly_cross_terms(x_tr2,0,False,False,False)
x_tr3_augment = build_poly_cross_terms(x_tr3,0,False,False,False)

print(x_tr0.shape)
print(x_tr0_augment.shape)
print(x_tr0_augment[1, :])

(73790, 23)
(73790, 276)
[ 1.00004000e+05 -1.00000000e+00  1.75864000e+02  1.69150000e+01
  1.34805000e+02  1.64050000e+01 -9.99000000e+02  3.89100000e+00
  1.64050000e+01  5.79830000e+01  1.05600000e+00 -1.38500000e+00
  2.82090000e+01 -2.19700000e+00 -2.23100000e+00  2.97740000e+01
  7.98000000e-01  1.56900000e+00  0.00000000e+00 -9.99000000e+02
 -9.99000000e+02 -9.99000000e+02  0.00000000e+00 -1.00004000e+05
  1.75871035e+07  1.69156766e+06  1.34810392e+07  1.64056562e+06
 -9.99039960e+07  3.89115564e+05  1.64056562e+06  5.79853193e+06
  1.05604224e+05 -1.38505540e+05  2.82101284e+06 -2.19708788e+05
 -2.23108924e+05  2.97751910e+06  7.98031920e+04  1.56906276e+05
  0.00000000e+00 -9.99039960e+07 -9.99039960e+07 -9.99039960e+07
  0.00000000e+00 -1.75864000e+02 -1.69150000e+01 -1.34805000e+02
 -1.64050000e+01  9.99000000e+02 -3.89100000e+00 -1.64050000e+01
 -5.79830000e+01 -1.05600000e+00  1.38500000e+00 -2.82090000e+01
  2.19700000e+00  2.23100000e+00 -2.97740000e+01 -7.98000000e-01




# Least square 

In [21]:
loss_mse, w = least_squares(yb,tx)
print(w.shape,"\n",loss_mse)

(30,) 
 0.7022239450685107


## least square for jet categories

In [22]:
loss_mse, w = least_squares(x_tr0[:, 1], x_tr0[:, 2:])
print(w.shape,"\n",loss_mse)

LinAlgError: Singular matrix

## Ridge Regression

In [23]:
lmbd = 0.001
loss_mse, w = ridge_regression(yb,tx,lmbd)
print(w.shape,"\n",loss_mse)

(30,) 
 0.7023654887402898


## Ridge Regression for jet categories

In [24]:
# Start ML algorithm.
lmbd = 0
loss0, w0 = ridge_regression(x_tr0_augment[:, 1], x_tr0_augment[:, 2:], lmbd)
loss1, w1 = ridge_regression(x_tr1_augment[:, 1], x_tr1_augment[:, 2:], lmbd)
loss2, w2 = ridge_regression(x_tr2_augment[:, 1], x_tr2_augment[:, 2:], lmbd)
loss3, w3 = ridge_regression(x_tr3_augment[:, 1], x_tr3_augment[:, 2:], lmbd)
#loss0_wm, w0_wm = ridge_regression(x_tr0_wm[:, 1], x_tr0_wm[:, 2:], lmbd)
#loss1_wm, w1_wm = ridge_regression(x_tr1_wm[:, 1], x_tr1_wm[:, 2:], lmbd)
#loss2_wm, w2_wm = ridge_regression(x_tr2_wm[:, 1], x_tr2_wm[:, 2:], lmbd)
#loss3_wm, w3_wm = ridge_regression(x_tr3_wm[:, 1], x_tr3_wm[:, 2:], lmbd)

print(w0.shape,"\n",loss0)
print(w1.shape,"\n",loss1)
print(w2.shape,"\n",loss2)
print(w3.shape,"\n",loss3)
  

LinAlgError: Singular matrix

## Cross validation

In [139]:
folds_n = 10
validation_scores = []

for i in range(0, folds_n):
    print("================ step " + str(i + 1) + " ================")

    x_tr, x_te, y_tr, y_te = split_data(txn_augment, yb, 1 / folds_n)
    # Start ML algorithm.
    # loss, w = least_squares_GD(y_tr, x_tr, initial_w, max_iterations, gamma)
    w, loss = least_squares(y_tr,x_tr)

    # Print result
    # print("Gradient Descent: execution time={t:.3f} seconds".format(t=execution_time))
    # print("================ loss ================")
    # print(loss)
    # print("================ w ================")
    # print(w)

    # Test algorithm
    y_te_predicted = predict_labels(w, x_te)
    score = validate(y_te_predicted, y_te)

    # Use SKLearn
    # clf = SGDClassifier(loss="squared_loss", penalty="l2", max_iter=20)
    # clf.fit(x_tr, y_tr)
    # print(clf)
    # y_te_predicted_sk = clf.predict(x_te)
    # score = validate(y_te_predicted_sk, y_te)


    validation_scores.append(score)
    print("Accuracy score:" + str(score))
    
    
    
cv_score = np.mean(np.array(validation_scores))
print("================ Final validation Score ================")
print("MEAN-Accuracy score:" + str(cv_score))

================ step 1 ================
Accuracy score:0.7080977777777778
================ step 2 ================
Accuracy score:0.7237777777777777
================ step 3 ================
Accuracy score:0.7296044444444445
================ step 4 ================
Accuracy score:0.7292177777777777
================ step 5 ================
Accuracy score:0.7388088888888888
================ step 6 ================
Accuracy score:0.6614311111111111
================ step 7 ================
Accuracy score:0.5956311111111111
================ step 8 ================
Accuracy score:0.7598488888888889
================ step 9 ================
Accuracy score:0.7426977777777778
================ step 10 ================
Accuracy score:0.7072044444444444
================ Final validation Score ================
MEAN-Accuracy score:0.709632


## load test data, create labels

In [104]:
test_data_path = "data/test.csv"
test_yb, test_tx, test_ids = load_csv_data(test_data_path)

## preprocess

In [105]:
test_tx_clean = pre_process_data(test_tx)

0---112.541
1---46.467
2---73.74
3---38.472
4---2.099
5---226.345
6----0.244
7---2.492
8---12.413
9---120.666
10---1.282
11----0.356
12---0.456
13---31.7655
14----0.022
15----0.042
16---40.553
17----0.038
18---0.097
19---34.754
20----0.016
21---179.94
22---1.0
23---65.839
24---0.001
25----0.048
26---48.037
27----0.012
28----0.027
29---40.504


In [149]:
test_txn_augment = np.concatenate((test_tx_clean, test_tx_clean),1)
print(test_txn_augment.shape)

test_txn_augment[:,30:60] = np.square(test_tx_clean)

(568238, 60)


In [150]:
test_yb = predict_labels(w, test_txn_augment)
print("================ labels ================")
print(test_yb[:50])

================ labels ================
[ 1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1.]


## create submission

In [151]:
create_csv_submission(test_ids, test_yb, "ls_predictions_complete_preprocess_augmented2")